In [1]:
%load_ext zipline
%load_ext watermark

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import zipline
from yahoofinancials import YahooFinancials
import warnings
import pandas_datareader as pdr
plt.style.use('seaborn')

plt.rcParams['figure.figsize'] = [16, 9]
plt.rcParams['figure.dpi'] = 200
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
watermark --iversions

### There are two methods to use a custom dataset, one is shown below which converts dataframe into panel, the other one using zipline bundle ingest, check 'Zipline Local Pipeline' notebook for that.

In [5]:
import pandas as pd
from collections import OrderedDict
import pytz
import pandas_datareader as pdr

def get_from_pdr_to_panel(symbol="NVDA", datefrom="2016-01-01", dateto="2018-01-01"):
    nvidia=pdr.get_data_yahoo(symbol, datefrom, dateto)
    nvidia.head()
    data = OrderedDict()
    data[symbol] = nvidia
    data[symbol] = data[symbol][["Open","High","Low","Close","Volume"]]
    #print(data[symbol].head())
    panel = pd.Panel(data)
    panel.minor_axis = ["open","high","low","close","volume"]
    panel.major_axis = panel.major_axis.tz_localize(pytz.utc)
    print(panel)
    return panel

In [10]:
from scripts.get_from_pdr_to_panel import get_from_pdr_to_panel

panel = get_from_pdr_to_panel()

<class 'pandas.core.panel.Panel'>
Dimensions: 1 (items) x 503 (major_axis) x 5 (minor_axis)
Items axis: NVDA to NVDA
Major_axis axis: 2016-01-04 00:00:00+00:00 to 2017-12-29 00:00:00+00:00
Minor_axis axis: open to volume


E:\anaconda3\envs\env_zipline\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
from zipline.api import order, record, symbol, set_benchmark
import zipline
import matplotlib.pyplot as plt
from datetime import datetime


def initialize(context):
    set_benchmark(symbol("NVDA"))


def handle_data(context, data):
    order(symbol("NVDA"), 10)
    record(NVDA=data.current(symbol('NVDA'), 'price'))

perf = zipline.run_algorithm(start=datetime(2016, 1, 5, 0, 0, 0, 0, pytz.utc),
                      end=datetime(2017, 12, 29, 0, 0, 0, 0, pytz.utc),
                      initialize=initialize,
                      capital_base=100000,
                      handle_data=handle_data,
                      data=panel)

In [9]:
perf.portfolio_value.pct_change().fillna(0).add(1).cumprod().sub(1)

2016-01-05 21:00:00+00:00    0.000000
2016-01-06 21:00:00+00:00   -0.000002
2016-01-07 21:00:00+00:00   -0.000128
2016-01-08 21:00:00+00:00   -0.000260
2016-01-11 21:00:00+00:00   -0.000246
2016-01-12 21:00:00+00:00   -0.000048
2016-01-13 21:00:00+00:00   -0.000510
2016-01-14 21:00:00+00:00   -0.000865
2016-01-15 21:00:00+00:00   -0.001959
2016-01-19 21:00:00+00:00   -0.001784
2016-01-20 21:00:00+00:00   -0.001651
2016-01-21 21:00:00+00:00   -0.001332
2016-01-22 21:00:00+00:00   -0.000619
2016-01-25 21:00:00+00:00   -0.000656
2016-01-26 21:00:00+00:00   -0.000294
2016-01-27 21:00:00+00:00   -0.000771
2016-01-28 21:00:00+00:00   -0.001238
2016-01-29 21:00:00+00:00    0.000745
2016-02-01 21:00:00+00:00    0.000760
2016-02-02 21:00:00+00:00   -0.001473
2016-02-03 21:00:00+00:00   -0.001209
2016-02-04 21:00:00+00:00   -0.001190
2016-02-05 21:00:00+00:00   -0.004930
2016-02-08 21:00:00+00:00   -0.007593
2016-02-09 21:00:00+00:00   -0.006973
2016-02-10 21:00:00+00:00   -0.007119
2016-02-11 2

In [41]:
data = OrderedDict()
data['NVDA'] = nvidia
data['NVDA'] = data['NVDA'][["Open","High","Low","Close","Volume"]]
data['NVDA'] = data['NVDA'].resample("1d").mean()
data['NVDA'].fillna(method="ffill", inplace=True)
print(data['NVDA'].head())

                 Open       High        Low      Close      Volume
Date                                                              
2016-01-04  32.290001  32.580002  32.040001  32.369999   8951900.0
2016-01-05  32.980000  33.439999  32.500000  32.889999  12256800.0
2016-01-06  32.349998  32.500000  31.160000  31.530001  11233600.0
2016-01-07  30.740000  30.950001  29.879999  30.280001  16132600.0
2016-01-08  30.670000  30.700001  29.570000  29.629999   9961800.0


In [43]:
import matplotlib.pyplot as plt
from matplotlib import style


style.use("ggplot")

perf.portfolio_value.pct_change().fillna(0).add(1).cumprod().sub(1).plot(label='portfolio')
perf.NVDA.pct_change().fillna(0).add(1).cumprod().sub(1).plot(label='benchmark')
plt.legend(loc=2)

plt.show()

In [8]:
perf

NVDA  algo_volatility  \
2016-01-05 21:00:00+00:00   32.889999              NaN   
2016-01-06 21:00:00+00:00   31.530001         0.000019   
2016-01-07 21:00:00+00:00   30.280001         0.001153   
2016-01-08 21:00:00+00:00   29.629999         0.001176   
2016-01-11 21:00:00+00:00   29.680000         0.001161   
2016-01-12 21:00:00+00:00   30.180000         0.001912   
2016-01-13 21:00:00+00:00   29.260000         0.003233   
2016-01-14 21:00:00+00:00   28.670000         0.003388   
2016-01-15 21:00:00+00:00   27.110001         0.006106   
2016-01-19 21:00:00+00:00   27.330000         0.006085   
2016-01-20 21:00:00+00:00   27.480000         0.005963   
2016-01-21 21:00:00+00:00   27.799999         0.006078   
2016-01-22 21:00:00+00:00   28.450001         0.006861   
2016-01-25 21:00:00+00:00   28.420000         0.006592   
2016-01-26 21:00:00+00:00   28.700001         0.006570   
2016-01-27 21:00:00+00:00   28.360001         0.006603   
2016-01-28 21:00:00+00:00   28.049999         0.006593   
2016-01-29 21:00:00+00:00   29.290001         0.010009   
2016-02-01 21:00:00+00:00   29.299999         0.009728   
2016-02-02 21:00:00+00:00   28.059999         0.012437   
2016-02-03 21:00:00+00:00   28.200001         0.012179   
2016-02-04 21:00:00+00:00   28.209999         0.011888   
2016-02-05 21:00:00+00:00   26.430000         0.016855   
2016-02-08 21:00:00+00:00   25.219999         0.018314   
2016-02-09 21:00:00+00:00   25.490000         0.018176   
2016-02-10 21:00:00+00:00   25.430000         0.017813   
2016-02-11 21:00:00+00:00   25.299999         0.017468   
2016-02-12 21:00:00+00:00   25.730000         0.017649   
2016-02-16 21:00:00+00:00   26.990000         0.020397   
2016-02-17 21:00:00+00:00   27.660000         0.020848   
...                               ...              ...   
2017-11-16 21:00:00+00:00  211.610001         0.499711   
2017-11-17 21:00:00+00:00  211.360001         0.499204   
2017-11-20 21:00:00+00:00  214.080002         0.498806   
2017-11-21 21:00:00+00:00  216.050003         0.498331   
2017-11-22 21:00:00+00:00  214.929993         0.497890   
2017-11-24 18:00:00+00:00  216.960007         0.497422   
2017-11-27 21:00:00+00:00  214.139999         0.497220   
2017-11-28 21:00:00+00:00  210.710007         0.497155   
2017-11-29 21:00:00+00:00  196.419998         0.503111   
2017-11-30 21:00:00+00:00  200.710007         0.503116   
2017-12-01 21:00:00+00:00  197.679993         0.503022   
2017-12-04 21:00:00+00:00  186.660004         0.507248   
2017-12-05 21:00:00+00:00  187.740005         0.506742   
2017-12-06 21:00:00+00:00  189.259995         0.506270   
2017-12-07 21:00:00+00:00  191.990005         0.505967   
2017-12-08 21:00:00+00:00  191.490005         0.505484   
2017-12-11 21:00:00+00:00  194.660004         0.505263   
2017-12-12 21:00:00+00:00  190.839996         0.505446   
2017-12-13 21:00:00+00:00  186.179993         0.505991   
2017-12-14 21:00:00+00:00  186.470001         0.505476   
2017-12-15 21:00:00+00:00  191.559998         0.505934   
2017-12-18 21:00:00+00:00  197.899994         0.506829   
2017-12-19 21:00:00+00:00  196.110001         0.506502   
2017-12-20 21:00:00+00:00  196.800003         0.505992   
2017-12-21 21:00:00+00:00  195.889999         0.505552   
2017-12-22 21:00:00+00:00  195.270004         0.505087   
2017-12-26 21:00:00+00:00  197.440002         0.504693   
2017-12-27 21:00:00+00:00  197.169998         0.504207   
2017-12-28 21:00:00+00:00  197.399994         0.503705   
2017-12-29 21:00:00+00:00  193.500000         0.503911   

                           algorithm_period_return     alpha  \
2016-01-05 21:00:00+00:00                 0.000000       NaN   
2016-01-06 21:00:00+00:00                -0.000002 -0.000118   
2016-01-07 21:00:00+00:00                -0.000128 -0.004867   
2016-01-08 21:00:00+00:00                -0.000260 -0.010506   
2016-01-11 21:00:00+00:00                -0.000246 -0.006170   
2016-01-12 21:00:00+00:00                -0.000048  